# ESAT 2022 - FeO$_\text{s}$ demonstration

**You can find this notebook in the `feos-org/demo-notebooks/` repositiory on github.**

In [ ]:
#@title Download FeO$_\text{s}$ and parameter files
#!pip install feos
!wget https://raw.githubusercontent.com/feos-org/feos/main/parameters/pcsaft/loetgeringlin2018.json

In [ ]:
#@title Import Python packages

from feos.si import *     # SI units and constants
from feos.eos import *    # equation of state objects
from feos.pcsaft import * # parameter objects

import numpy as np              # for arrays
import matplotlib.pyplot as plt # plotting utilities
import seaborn as sns
sns.set_style("ticks")
sns.set_context("poster")
sns.set_palette("Dark2")

## Python Class as Equation of State

In [ ]:
#@title Python class

class PengRobinson:
    def __init__(self, critical_temperature, critical_pressure, acentric_factor, molar_weight):
        self.tc = critical_temperature / KELVIN
        self.pc = critical_pressure / PASCAL
        self.omega = acentric_factor
        self.mw = molar_weight / GRAM * MOL
        self.a_r = 0.45724 * critical_temperature**2 * RGAS / critical_pressure / ANGSTROM**3 / NAV / KELVIN
        self.b = 0.07780 * critical_temperature * RGAS / critical_pressure / ANGSTROM**3 / NAV
        self.kappa = 0.37464 + (1.54226 - 0.26992 * acentric_factor) * acentric_factor
    
    def components(self) -> int: return 1
    def subset(self, i: [int]): return self
    def molar_weight(self) -> SIArray1: return SIArray1(self.mw * GRAM / MOL)
    def max_density(self, moles:[float]) -> float:
        b = np.sum(moles * self.b) / np.sum(moles)
        return 0.9 / b 
    
    def helmholtz_energy(self, state):
        print(f'Data type: {type(state.temperature)}')
        print(f'T: {state.temperature}')
        print(f'V: {state.volume}')
        print(f'N: {sum(state.moles)}')
        print('')
        
        ak = ((1.0 - np.sqrt(state.temperature / self.tc)) * self.kappa + 1.0)**2 * self.a_r
        rho = np.sum(state.partial_density)
        a = np.sum(state.moles) * (
            -np.log(1.0 - self.b * rho) 
            - ak / (self.b * np.sqrt(2.0) * 2.0 * state.temperature) 
                * np.log(
                    (np.sqrt(2.0) - 1.0 + self.b * rho) / 
                    (np.sqrt(2.0) + 1.0 - self.b * rho)
                )
        )
        return a

In [ ]:
# propane
pr = PengRobinson(
    critical_temperature=369.96 * KELVIN, 
    critical_pressure=42.5 * BAR, 
    acentric_factor=0.153, 
    molar_weight=44.0962 * GRAM / MOL
)
# Inject Python class into FeOs
eos_py = EquationOfState.python(pr)

In [ ]:
# define thermodynamic conditions
# here: natural variables of A
state = State(
    eos_py, 
    temperature=300*KELVIN, 
    volume=1e3 * ANGSTROM**3, 
    total_moles=1 / NAV
)
state

In [ ]:
state.molar_helmholtz_energy()
#state.pressure()
#state.molar_entropy()
#state.chemical_potential()[0]
#state.c_p();

## PC-SAFT

In [ ]:
#@title 1. Define Model
parameters = PcSaftParameters.from_json(
    ["hexane", "toluene"], 
    "loetgeringlin2018.json"
)
pcsaft = EquationOfState.pcsaft(parameters)

In [ ]:
#@title 2. Define Thermodynamic Conditions
state = State(
    pcsaft, 
    temperature=315.0 * KELVIN,
    pressure=2.0 * BAR,
    molefracs=np.array([0.5 , 0.5]),
    total_moles=150*MOL
)
state

In [ ]:
#@title 3. Compute Properties

state.chemical_potential() # array valued

## Critical Points & Phase Equilibria

In [ ]:
#@title State at Critical Conditions
state_cp = State.critical_point(
    pcsaft, 
    moles=np.array([1.0, 1.0]) * MOL
)
state_cp

In [ ]:
state_cp.pressure()

In [ ]:
#@title Compute phase diagram
phase_diagram = PhaseDiagram.binary_vle(
    pcsaft,
    350 * KELVIN,
    npoints=251
)

In [ ]:
#@title Plot phase diagram
KG_M3 = KILOGRAM / METER**3
plt.figure(figsize=(12, 7))
plt.title(f'hexane(1), toluene(2), T = 350 K')
plt.plot(
    phase_diagram.vapor.molefracs[:, 0],
    phase_diagram.vapor.pressure / BAR
)
plt.plot(
    phase_diagram.liquid.molefracs[:, 0],
    phase_diagram.liquid.pressure / BAR
)
plt.ylabel(r"$p$ / bar")
plt.xlabel(r"$x_1$")
plt.xlim(0, 1)
plt.ylim(0.2, 1.4);

In [ ]:
phase_diagram.states[0]

In [ ]:
#@title Entropy scaling
entropies = []
viscosities = []
x = np.array([0.3, 0.7])
for p in SIArray1.linspace(0.1 * BAR, 5 * BAR, 10):
    for x1 in np.linspace(0.1, 0.9, 10):
        state = State(pcsaft, 350 * KELVIN, pressure=p, molefracs=np.array([x1, 1-x1]))
        entropies.append(state.molar_entropy(Contributions.ResidualNvt) / RGAS)
        viscosities.append(state.ln_viscosity_reduced())
    
plt.figure(figsize=(12, 7))
plt.title('T = 350 K, p = [0.1, 5] bar, x1 = [0.1, 0.9]')
#plt.plot(sorted(entropies, reverse=True), sorted(viscosities), '-')
plt.plot(entropies, viscosities, 'o:')
plt.xlim(-6, 0.1)
plt.ylim(-1.5, 3)
plt.xlabel(r'$s^\mathrm{res} / R$')
plt.ylabel(r'$\ln \frac{\eta}{\eta^\mathrm{CE}}$');

## Classical Density Functional Theory (DFT)

In [ ]:
from feos.dft import *    # classical density function theory objects
parameters = PcSaftParameters.from_json(
    ["hexane", "toluene"], 
    "loetgeringlin2018.json"
)
func = HelmholtzEnergyFunctional.pcsaft(parameters)

In [ ]:
#@title Surface tension
#@markdown 1. compute VLE
#@markdown 2. set initial density profile
#@markdown 3. solve for equilibrium profile

vle = PhaseEquilibrium.tp_flash(
    func,
    500 * KELVIN,
    16 * BAR,
    np.array([0.3, 0.7]) * MOL
)
interface = PlanarInterface.from_tanh(
    vle, 1024, 100 * ANGSTROM, 500 * KELVIN
).solve()

In [ ]:
interface.surface_tension

In [ ]:
#@title Density profiles
KMOL_M3 = KILO * MOL / METER**3
plt.figure(figsize=(12, 7))
plt.plot(interface.z / ANGSTROM, (interface.density / KMOL_M3)[0], 
         label="{}".format(parameters.pure_records[0].identifier.name))
plt.plot(interface.z / ANGSTROM, (interface.density / KMOL_M3)[1],
         label="{}".format(parameters.pure_records[1].identifier.name))
plt.xlabel(r"$z$ / A");
plt.ylabel(r"$\rho$ / kmol / m$^3$")
plt.ylim(0, 5);
plt.xlim(0, 100)
plt.legend(frameon=False);

In [ ]:
#@title Adsorption in pore
potential = ExternalPotential.LJ93(3.0, 100.0, 0.08)

pore = Pore1D(
    geometry=Geometry.Cartesian, 
    pore_size=40 * ANGSTROM, 
    potential=potential
)

isotherm = Adsorption1D.equilibrium_isotherm(
    func,
    temperature=500 * KELVIN,
    pressure=SIArray1.linspace(6*BAR, 22.0001*BAR, 5),
    pore=pore,
    molefracs=np.array([0.5, 0.5])
)

In [ ]:
#@title Adsorpted amount per surface
plt.figure(figsize=(12, 7))
plt.title(r"$T$ = 500 K")
plt.plot(
    isotherm.pressure/BAR, 
    isotherm.total_adsorption/(MICRO*MOL/METER**2)
)
plt.xlim(6, 22)
plt.ylim(1, 12)
plt.xlabel(r'$p$ / bar')
plt.ylabel(r'$N$ /  $\mu$mol / m$^2$');

In [ ]:
#@title Density profiles
KMOL_M3 = KILO * MOL / METER**3
i = 6 #@param {type:"slider", min:0, max:6, step:1}
z = isotherm.profiles[0].z / ANGSTROM
p = isotherm.pressure / BAR
total_adsorption = isotherm.total_adsorption / (MICRO * MOL / METER**2)

# figure
fig, ax = plt.subplots(
    1, 2, figsize=(15, 5), 
    gridspec_kw={'wspace': 0.25}
)

# isotherm
ax[0].plot(p, total_adsorption)
ax[0].plot(p[i], total_adsorption[i], marker="s", clip_on=False)
ax[0].set_xlim(6, 22)
ax[0].set_xticks(range(6, 23, 4))
ax[0].set_ylim(1, 12)
ax[0].set_xlabel(r'$p$ / bar')
ax[0].set_ylabel(r'$N$ /  $\mu$mol / m$^2$');

# density profile
ax[1].plot(z, (isotherm.profiles[i].density / KMOL_M3)[0], 
         label="{}".format(parameters.pure_records[0].identifier.name))
ax[1].plot(z, (isotherm.profiles[i].density / KMOL_M3)[1], 
         label="{}".format(parameters.pure_records[1].identifier.name))
ax[1].set_xlim(0, 20)
ax[1].set_ylim(0, 12)
ax[1].set_xlabel(r"$z$ / A")
ax[1].set_ylabel(r"$\rho$ / kmol / m$^3$")
ax[1].legend(frameon=False, loc="upper left");